# PD 2
## Hubert Ruczyński

# Importy

In [ ]:
%matplotlib inline
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import sklearn
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
np.random.seed(23)
import category_encoders as ce

# ***Część I***

## Treść

Wykonaj target encoding dla zmiennej it_location. Czy i jakie są przewagi target encoding nad one-hot? Jako target traktujemy kolumnę price (będzie to więc zadanie regresji).

Zastosuj trzy metody encodingu (one-hot + "dwie nowe") dla kolumny main_category. "Nowe metody" proszę wybrać spośród wymienionych na stronie https://contrib.scikit-learn.org/category_encoders/. W przypadku, gdy użyta metoda nie działa proszę o stosowną adnotację. Opisz wyniki.

Zwizalizuj wynik oraz wyjaśnij czym się różnią sposoby kodowania (czemu to działa).

In [ ]:
transactions_df = pd.read_csv('allegro-api-transactions.csv', index_col='lp')
transactions_df.sample(3)

In [ ]:
transactions_df.info()

In [ ]:
len(transactions_df.loc[:,'it_location'].unique())

In [ ]:
transactions_df.head()

In [ ]:
print(transactions_df.isnull().sum())

In [ ]:
X=transactions_df.drop('price', axis =1)
y=transactions_df.drop('it_location', axis=1)

## Target Encoder

In [ ]:
encoder = ce.TargetEncoder(cols=['it_location'])
encoder.fit(X,y['price'])
df_target_encoder=encoder.transform(X,y['price'])
df_target_encoder.head(3)

In [ ]:
len(df_target_encoder.loc[:,'it_location'].unique())

In [ ]:
onehot = ce.OneHotEncoder(handle_unknown='ignore')
encoded_onehot = pd.DataFrame(onehot.fit_transform(transactions_df[["main_category"]]))
data_onehot = transactions_df.join(encoded_onehot)
data_onehot.head(3)

Inspirując się artykułem: https://medium.com/analytics-vidhya/target-encoding-vs-one-hot-encoding-with-simple-examples-276a7e7b3e64, udało mi się wyróżnić proste wady i zalety każdego z kodowań.

**OneHot**

*Zalety:*

Działa dobrze w przypadku danych nominalnych i eliminuje problem z wyższymi wartościami kategorycznymi wpływającymi na dane, ponieważ tworzymy każdą kolumnę w binarnej 1 lub 0.

*Wady:*

Może stworzyć bardzo dużą wymiarowość w zależności od liczby posiadanych cech kategorycznych i liczby kategorii na obiekt, co w ostateczności doprowadzić może do bardzo działania.

**Target**

*Opis:*

Polega na tym, że funkcje są zastępowane połączeniem późniejszego prawdopodobieństwa celu przy danej określonej wartości kategorycznej i wcześniejszego prawdopodobieństwa celu względem wszystkich danych uczących.

*Zalety:*

Prosta i szybka metoda kodowania, która nie zwiększa wymiarowości zbioru danych.

*Wady:*

Zależy ono od dystrybucji celu, co oznacza, że wymaga starannej weryfikacji, ponieważ może być podatne na overfitting. Ta metoda jest również specyficzna dla zbioru danych i będzie wykazywać znaczną poprawę tylko przez pewien czas.

## ***OneHotEncoding***

In [ ]:
X3=transactions_df.drop('price', axis =1)
y3=transactions_df.drop('main_category', axis =1)
ce_one_hot = ce.OneHotEncoder(cols = ['main_category'])
ce_one_hot_df=ce_one_hot.fit_transform(X3,y3)

In [ ]:
ce_one_hot_df.iloc[:, 11:38]

In [ ]:
OneHot=ce_one_hot_df.iloc[:, 11:38]
OneHot=OneHot.sum()
OneHot

In [ ]:
bars = ('main_category_1', 'main_category_2','main_category_3', 'main_category_4','main_category_5', 'main_category_6','main_category_7', 'main_category_8','main_category_9', 'main_category_10','main_category_11', 'main_category_12','main_category_13', 'main_category_14','main_category_15', 'main_category_16','main_category_17', 'main_category_18','main_category_19', 'main_category_20','main_category_21', 'main_category_22','main_category_23', 'main_category_24','main_category_25','main_category_26','main_category_27')
y_pos = np.arange(len(bars))
plt.bar(y_pos,OneHot)

Metoda OneHotEncoding działa w ten sposób, że rozdziela zadaną kolumnę na wszystkie unikalne kategorie (kolejne kolumny zamiast tamtej) i następnie dla każdej zmiennej daje 1 w miejscu gdzie chodzi o daną kategorię, a 0 jeśli nie.

## ***LeaveOneOut***

In [ ]:
X4=transactions_df.drop('price', axis =1)
y4=transactions_df.drop('main_category', axis =1)
ce_leave = ce.LeaveOneOutEncoder(cols = ['main_category'])
ce_leave.fit(X4, y4['price'])        
ce_leave_df=ce_leave.transform(X4, y4['price'])

In [ ]:
leave=ce_leave_df.iloc[:, 11:38]
leave

In [ ]:
sns.histplot(leave)

Kodowanie Leave One Out  oblicza średnią zmiennych docelowych dla wszystkich rekordów zawierających tę samą wartość dla danej zmiennej jakościowej. Algorytm kodowania różni się nieco w zestawie danych uczących i testowych. 

W przypadku zestawu danych uczących omawiany rekord jest pomijany, stąd nazwa Leave One Out. Kodowanie jest następujące dla pewnej wartości określonej zmiennej kategorialnej.

W przypadku danych walidacyjnych lub zestawu danych prognozowania definicja jest nieco inna. Nie musimy zostawiać aktualnego rekordu i nie potrzebujemy czynnika losowości.

## ***Helmert Encoding***

In [ ]:
X5=transactions_df.drop('price', axis =1)
y5=transactions_df.drop('main_category', axis =1)
ce_helmert = ce.HelmertEncoder(cols = ['main_category'])
ce_helmert_df=ce_helmert .fit_transform(X5, y5)

In [ ]:
helmert=ce_helmert_df.iloc[:, 13:39]
helmert

In [ ]:
helmert.max(axis=1)

In [ ]:
n_bins=27
plt.hist(helmert.max(axis=1), bins=n_bins)
plt.show()

Metoda Helmerta działa w sposób podobny do do OneHotEncoding, gdyż w miejscu w któym mamy do czynienia z n-tą kategorią wstawiana jest liczba n odpowiadająca danej kategorii. Pozostałe kategorie są obsługiwane w taki sposób, że te wcześniejsze otrzymują 0 zaś późniejsze 1.

Kodowanie Helmerta jest typem kodowania kategorialnego dla regresji. Porównuje każdy poziom zmiennej kategorialnej ze średnią kolejnych poziomów. Stąd pierwszy kontrast porównuje średnią zmiennej zależnej ze średnią wszystkich kolejnych poziomów kolumny kategorialnej, drugi kontrast porównuje średnią zmienna zależna ze średnią wszystkich kolejnych poziomów, itd.

# ***Część II***

## Treść

W tej części zadania traktujemy zmienną price nie jak target a zmienną objaśniającą. Zbiór danych ograniczamy do zmiennych numerycznych tj. price, it_seller_rating i it_quantity.

Proszę losowo usunąć 10% wartości ze zmiennej it_seller_rating i je uzupełnić z użyciem jednego z automatycznych narzędzi: Nearest neighbors imputation lub Multivariate feature imputation (https://scikit-learn.org/stable/modules/impute.html).
Następnie należy porównać wartości imputowane z oryginalnymi (polecam miarę RMSE). Eksperyment powtórzyć 10 razy i zobaczyć jakie będzie odchylenie standardowe wyniku. Następnie zrobić analogiczną analizę gdy oprócz losowego usuwania 10% wartości z kolumny it_seller_rating usuniemy także losowo 10% ze zmiennej it_quantity. (w przypadku problemów wydajnościowych proszę ograniczyć liczbę rekordów).

Opisać wnioski z analizy jakości imputacji i umieścić podsumowujący wykres.

## **Pojedyncza iteracja**

Najpierw dokonajmy pojedynczej iteracji dla opisanego zadania z usuwaniem danych tylko z jednej kolumny.

In [ ]:
df=transactions_df[['price', 'it_seller_rating', 'it_quantity']]
df2=df.copy()
df2

In [ ]:
df2.loc[df.sample(frac=0.1).index, 'it_seller_rating'] = np.nan
df2

In [ ]:
df2.isnull().sum()

Do imputacji brakujących zmiennych używam IterativeImputera (Multivariate feature imputation)

In [ ]:
imp = IterativeImputer(random_state=0, skip_complete=True, n_nearest_features=10)
df3=imp.fit_transform(df2)
df3=pd.DataFrame(df3, columns=['price','it_seller_rating','it_quantity'])
df3

In [ ]:
df3.isnull().sum()

Imputer zadziałał, gdyż nie mamy brakujących danych.

In [ ]:
sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1],squared=False)

In [ ]:
np.std(df, ddof=0)

In [ ]:
np.std(df3, ddof=0)

## 10 iteracji dla jednej kategorii wybrakowanych

In [ ]:
imp = IterativeImputer(max_iter=10000, random_state=10, n_nearest_features=100,initial_strategy='mean')
v1 = []
for i in range(10):
    df2=df.copy()
    df2.loc[df.sample(frac=0.1).index, 'it_seller_rating'] = np.nan
    df3=imp.fit_transform(df2)
    df3=pd.DataFrame(df3, columns=['price','it_seller_rating','it_quantity'])
    v1.append(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False))
    print("Mean squared Error : " + str(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False)))
    print('Standard deviation for all variables:')
    print(np.std(df3, ddof=0))
    print(" ")
std1 = np.std(v1, axis=0)
print('Standard deviation for RMSE: '+str(std1))

## 10 iteracji dla dwóch kategorii wybrakowanych

In [ ]:
imp = IterativeImputer(max_iter=10000, random_state=10, n_nearest_features=100,initial_strategy='mean')
v2 = []
v3 = []
for i in range(10):
    df2=df.copy()
    df2.loc[df.sample(frac=0.1).index, 'it_seller_rating'] = np.nan
    df2.loc[df.sample(frac=0.1).index, 'it_quantity'] = np.nan
    df3=imp.fit_transform(df2)
    df3=pd.DataFrame(df3, columns=['price','it_seller_rating','it_quantity'])
    v2.append(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False))
    v3.append(sklearn.metrics.mean_squared_error(df.iloc[:, 2],df3.iloc[:, 2], squared=False))
    print("Mean squared Error it_seller_rating: " + str(sklearn.metrics.mean_squared_error(df.iloc[:, 1],df3.iloc[:, 1], squared=False)))
    print("Mean squared Error it_quantity: " + str(sklearn.metrics.mean_squared_error(df.iloc[:, 2],df3.iloc[:, 2], squared=False)))
    print('Standard deviation for all variables:')
    print(np.std(df3, ddof=0))
    print(" ")
std2 = np.std(v2, axis=0)
print('Standard deviation for it_seller_rating RMSE: '+str(std2))
std3 = np.std(v3, axis=0)
print('Standard deviation for it_quantity RMSE: '+str(std3))

## Wnioski po imputacji:

W obu przypadkach imputacja jest wysoce nieskuteczna, gdyż błąd średniokwadratowy jest duży.

Widoczne są też zmiany w wariancjach zmiennych które były uzupełniane.

Z drugiej strony stron, w przypadku braków danych, jest to coś z czego faktycznie opłaca się korzystać, gdyż jesteśmy w stanie pozyskać jedynie w małym,stopniu zaburzone rekordy, za to w dużej ilości.

In [ ]:
itera=[1,2,3,4,5,6,7,8,9,10]
iters=[0,1,2,3,4,5,6,7,8,9]

## Wizualizacje RMSE

In [ ]:
sns.set(style="whitegrid", color_codes=True)
fig, ax = plt.subplots() 
sns.barplot(itera, y=v1, color='blue', saturation=0.5)
ax.set_ylim(11000,12000)
plt.title('RMSE for first test')
plt.show()

In [ ]:
sns.set(style="whitegrid", color_codes=True)
fig, ax = plt.subplots() 
sns.barplot(itera, y=v2, color='cyan')
ax.set_ylim(11000,12000)
plt.title('RMSE for second test, middle column')
plt.show() 

In [ ]:
sns.set(style="whitegrid", color_codes=True)
fig, ax = plt.subplots() 
sns.barplot(itera, y=v3, color='darkblue')
ax.set_ylim(7000,7600)
plt.title('RMSE for second test, last column')
plt.show() 